In [1]:
import os
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

In [2]:
from typing import List, Dict, Any
import tensorflow as tf
import numpy as np

2025-12-26 00:33:37.192098: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-12-26 00:33:37.218397: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-12-26 00:33:37.225207: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-26 00:33:37.242512: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-26 00:33:38.820221: W tensorflow/compiler/tf2

## Types of Learning Schedules

### Cosine Warmup Learning Rate

$
\eta_t = 
\begin{cases} 
\eta_{max} \cdot \left(\frac{t}{t_{warmup}}\right) & \text{if } t < t_{warmup} \\
\eta_{min} + \frac{1}{2}(\eta_{max} - \eta_{min})\left(1 + \cos\left(\pi \cdot \frac{t - t_{warmup}}{t_{max} - t_{warmup}}\right)\right) & \text{otherwise} 
\end{cases}
$

In [3]:
class CosineWarmupSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, base_learning_rate: float, minimum_learning_rate: float, warmup_steps: int, total_steps:int):
        super(CosineWarmupSchedule, self).__init__()
        
        self.base_learning_rate = tf.constant(base_learning_rate, dtype= tf.float32)
        self.minimum_learning_rate = tf.constant(minimum_learning_rate, dtype= tf.float32)
        self.warmup_steps = tf.constant(warmup_steps, dtype= tf.int64)
        self.total_steps = tf.constant(total_steps, dtype= tf.int64)
        self.pi = tf.constant(3.141592653589793, tf.float32)
        
    def __call__(self, step: tf.Tensor):
        
        step = tf.cast(step, dtype = tf.float32)

        warmup_steps = tf.cast(self.warmup_steps, dtype = tf.float32)
        total_steps = tf.cast(self.total_steps, dtype = tf.float32)

        # Phase 1: Warm up learning rate
        warmup_learning_rate = self.base_learning_rate * tf.minimum(1.0, step / tf.maximum(1.0, warmup_steps))

        # Phase 2: Cosine Decay
        diff_in_steps = tf.maximum(1.0,total_steps - warmup_steps)
        progress = tf.clip_by_value((step - warmup_steps)/diff_in_steps, 0.0, 1.0)
        cosine_decay = self.minimum_learning_rate + 0.5 * (self.base_learning_rate - self.minimum_learning_rate) * (1 + tf.math.cos(self.pi * progress))

        return tf.where(step < warmup_steps, warmup_learning_rate, cosine_decay)       

In [4]:
cosine_decay = CosineWarmupSchedule(base_learning_rate = 0.01, minimum_learning_rate = 0.0001, warmup_steps = 1000, total_steps = 10000)

I0000 00:00:1766727220.610575    2529 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1766727220.715037    2529 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1766727220.715105    2529 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1766727220.716817    2529 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1766727220.716894    2529 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

In [5]:
cosine_decay(10000)

<tf.Tensor: shape=(), dtype=float32, numpy=1e-04>

### Constant Learning Rate

In [6]:
class ConstantLearningSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, base_learning_rate: float):
        super(ConstantLearningSchedule, self).__init__()
        
        self.base_learning_rate = tf.constant(base_learning_rate, dtype= tf.float32)
        
    def __call__(self, step: tf.Tensor):
        return self.base_learning_rate      

In [7]:
constant_lr = ConstantLearningSchedule(base_learning_rate = 0.01)

In [8]:
constant_lr(10)

<tf.Tensor: shape=(), dtype=float32, numpy=0.01>

### Step Decay Learning Rate

$
\text{lr}=\text{lr}_{\text{initial}}\times \gamma^{n(t)}
$

In [9]:
class StepDecaySchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, base_learning_rate: float, gamma: float, milestones: list[int]):
        super(StepDecaySchedule, self).__init__()
        
        self.base_learning_rate = tf.constant(base_learning_rate, dtype= tf.float32)
        self.gamma = tf.constant(gamma, dtype= tf.float32)
        self.milestones = tf.constant(milestones, dtype= tf.int64)
        
    def __call__(self, step: tf.Tensor):
        
        step = tf.cast(step, dtype = tf.int64)

        # Calculating the number of milestones passed
        num_milestones = tf.reduce_sum(tf.cast(step >= self.milestones, dtype= tf.float32))

        # Calculating the learning rate
        learning_rate = self.base_learning_rate * tf.math.pow(self.gamma, num_milestones)

        return learning_rate             

In [10]:
step_decay = StepDecaySchedule(base_learning_rate = 0.01, gamma = 0.1, milestones = [10000, 20000])

In [11]:
step_decay(20000)

<tf.Tensor: shape=(), dtype=float32, numpy=1e-04>

### Exponential Decay Learning Rate

$
\text{lr}=\text{lr}_{\text{initial}}\times \gamma^{\frac{t}{k}}
$

In [12]:
class ExponentialDecaySchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, base_learning_rate: float, gamma: float, decay_interval: int):
        super(ExponentialDecaySchedule, self).__init__()
        
        self.base_learning_rate = tf.constant(base_learning_rate, dtype= tf.float32)
        self.gamma = tf.constant(gamma, dtype= tf.float32)
        self.decay_interval = tf.constant(decay_interval, dtype= tf.int64)
        
    def __call__(self, step: tf.Tensor):
        
        step = tf.cast(step, dtype = tf.int64)

        # Calculating the decay factor
        decay_factor = tf.math.pow(self.gamma, tf.cast(step/self.decay_interval, dtype = tf.float32))

        # Calculating the exponential decay
        learning_rate = self.base_learning_rate * decay_factor

        return learning_rate          

In [13]:
exponential_decay = ExponentialDecaySchedule(base_learning_rate = 0.01, gamma = 0.9, decay_interval = 1000)

In [14]:
exponential_decay(10000)

<tf.Tensor: shape=(), dtype=float32, numpy=0.0034867835>

## Learning Rate Schedule Factory

In [21]:
class LearningRateScheduleFactory:
    @staticmethod
    def build(config: dict[str, Any]):
        lr_schedule_config = LearningRateScheduleFactory._create_scheduler_config_from_main_config(config)

        lr_schedule = LearningRateScheduleFactory._build_learning_schedule(lr_schedule_config)

        return lr_schedule
        
    @staticmethod
    def _create_scheduler_config_from_main_config(config: dict[str,Any]):
        scheduler = config['train'].get('scheduler')
        warmup_config = scheduler.get('warmup', {})
        step_config = scheduler.get('step', {})
        multistep_config = scheduler.get('multistep', {})
    
        schedule_config = {
            'learning_schedule': scheduler.get('name',"constant"),
            'base_learning_rate': scheduler.get('base_lr', 0.1),
            'mininum_learning_rate': scheduler.get('min_lr', 0.001),
            'total_steps': scheduler.get('total_steps', None),
            'warmup_epochs': warmup_config.get('epochs',10),
            'warmup_steps': warmup_config.get('steps', 15),
            'warmup_enabled': warmup_config.get('enabled',True),
            'warmup_start_factor': warmup_config.get('start_factor',0.1),
            'warmup_end_factor': warmup_config.get('end_factor',0.1),
            'warmup_mode': warmup_config.get('mode',0.1),
            'step_drop_every_epochs': step_config.get('drop_every_epochs', None),
            'step_gamma': step_config.get('gamma', None),
            'multistep_milestones': multistep_config.get('milestones_epochs', [])
        }
        
        return schedule_config
    
    @staticmethod
    def _build_learning_schedule(config: dict[str,Any]):
        # Need to select the correct learning schedule
        if config['learning_schedule'] == 'cosine_warmup':
            learning_schedule = CosineWarmupSchedule(base_learning_rate = config['base_learning_rate'], minimum_learning_rate = config['mininum_learning_rate'], warmup_steps = config['warmup_steps'], total_steps = config['total_steps'])
        elif config['learning_schedule'] == 'step_decay':
            learning_schedule = StepDecaySchedule(base_learning_rate = config['base_learning_rate'], gamma = config['step_gamma'], milestones = config['multistep_milestones'])
        elif config['learning_schedule'] == 'exponential_decay':
            learning_schedule = ExponentialDecaySchedule(base_learning_rate = config['base_learning_rate'], gamma = config['step_gamma'], decay_interval = config['step_drop_every_epochs'])
        elif config['learning_schedule'] == 'constant':
            learning_schedule = ConstantLearningSchedule(base_learning_rate = config['base_learning_rate'])
        else:
            raise ValueError(f"Invalid Learning Rate Schedule: {config['learning_schedule']}")

        return learning_schedule

## Factory Pattern

In [16]:
from mobilenetv2ssd.core.config import load_config

In [17]:
main_cfg_path = "configs/train/default.yaml"
model_cfg_path = "configs/model/mobilenetv2_ssd_voc.yaml"
data_cfg_path = "configs/data/voc_224.yaml"
eval_cfg_path = "configs/eval/default.yaml"

In [18]:
config = load_config(main_cfg_path,model_cfg_path,data_cfg_path,eval_cfg_path)

In [19]:
config['train']['scheduler']

{'interval': 'step',
 'name': 'cosine_warmup',
 'base_lr': 0.001,
 'min_lr': 1e-05,
 'total_steps': 10000,
 'warmup': {'enabled': True,
  'epochs': 5,
  'steps': 10,
  'start_factor': 0.1,
  'end_factor': 1.0,
  'mode': 'linear'},
 'step': {'drop_every_epochs': 10, 'gamma': 0.1},
 'multistep': {'milestones_epochs': [30, 45], 'gamma': 0.1}}

In [22]:
lr = LearningRateScheduleFactory.build(config)

In [23]:
lr